Business Understanding

The boss man wants safe planes. 

Data Understanding

In [2]:
import pandas as pd
import numpy as np

In [159]:
ad = pd.read_csv('Aviation_Data.csv') #named dataset "ad" for simplicity

In [161]:
ad.head() #see what the dataset looks like


,Event.Id,Investigation.Type,Accident.Number,Event.Date,Location,Country,Latitude,Longitude,Airport.Code,Airport.Name,...,Purpose.of.flight,Air.carrier,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,Weather.Condition,Broad.phase.of.flight,Report.Status,Publication.Date
0,20001218X45444,Accident,SEA87LA080,1948-10-24,"MOOSE CREEK, ID",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,2.0,0.0,0.0,0.0,UNK,Cruise,Probable Cause,NaN
1,20001218X45447,Accident,LAX94LA336,1962-07-19,"BRIDGEPORT, CA",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,4.0,0.0,0.0,0.0,UNK,Unknown,Probable Cause,19-09-1996
2,20061025X01555,Accident,NYC07LA005,1974-08-30,"Saltville, VA",United States,36.9222,-81.8781,NaN,NaN,...,Personal,NaN,3.0,NaN,NaN,NaN,IMC,Cruise,Probable Cause,26-02-2007
3,20001218X45448,Accident,LAX96LA321,1977-06-19,"EUREKA, CA",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,2.0,0.0,0.0,0.0,IMC,Cruise,Probable Cause,12-09-2000
4,20041105X01764,Accident,CHI79FA064,1979-08-02,"Canton, OH",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,1.0,2.0,NaN,0.0,VMC,Approach,Probable Cause,16-04-1980


In [162]:
ad = ad.drop_duplicates() #drop duplicate values

In [163]:
"""
Here I'm going to drop all of the columns that aren't of interest to me.  I'm interested in 
knowing which make/model, type of engine, and number of engines has the least amount of 
aircraft damage, injuries, and least severe injuries. 
"""

ad = ad.drop('Event.Id', axis = 1) 
ad = ad.drop('Investigation.Type', axis = 1) 
ad = ad.drop('Event.Date', axis =1)
ad = ad.drop('Location', axis = 1)
ad = ad.drop('Country', axis = 1)
ad = ad.drop('Latitude', axis = 1)
ad = ad.drop('Airport.Code', axis = 1)
ad = ad.drop('Airport.Name', axis = 1)
ad = ad.drop('Aircraft.Damage', axis = 1)
ad = ad.drop('Registration.Number', axis = 1)
ad = ad.drop('FAR.Description', axis = 1)
ad = ad.drop('Schedule', axis = 1)
ad = ad.drop('Purpose.of.flight', axis = 1)
ad = ad.drop('Air.carrier', axis = 1)
ad = ad.drop('Weather.Condition', axis = 1)
ad = ad.drop('Longitude', axis = 1)
ad = ad.drop('Broad.phase.of.flight', axis = 1)
ad = ad.drop('Report.Status', axis = 1)
ad = ad.drop('Publication.Date', axis = 1)
ad = ad.drop('Injury.Severity', axis = 1)
ad = ad.drop('Make', axis = 1)
ad = ad.drop('Model', axis = 1)


In [164]:
### Now we are going to drop all of events that happened in amateur-built airplanes because
### I'm not going to make a recommendation to my company to buy amateur-built airplanes. 

ad = ad[ad['Amateur.Built'] == 'No']
        

In [165]:
#Now I dropped the Amateur Built Column because all of the data is non-amateur-built airplanes. 

ad = ad.drop('Amateur.Built', axis = 1)

In [166]:
### Now I have all of the columns I want to do my analysis so I'm going to check and see if those
### columns have any missing data. 

ad.isna().sum()

Accident.Number               0
Aircraft.damage            3147
Aircraft.Category         51591
Number.of.Engines          5706
Engine.Type                6664
Total.Fatal.Injuries      10157
Total.Serious.Injuries    11084
Total.Minor.Injuries      10557
Total.Uninjured            4887
dtype: int64

In [167]:
""" The injuries seem to have a large amount of missing data but it looks like if there were no
injuries that they left the cell blank so I'm going to replace all of these missing data with 0's"""

ad['Total.Fatal.Injuries'] = ad['Total.Fatal.Injuries'].fillna(value=0)
ad['Total.Serious.Injuries'] = ad['Total.Serious.Injuries'].fillna(value=0)
ad['Total.Minor.Injuries'] = ad['Total.Minor.Injuries'].fillna(value=0)
ad['Total.Uninjured'] = ad['Total.Uninjured'].fillna(value=0)

In [168]:
### the rest of the missing data makes up about 8% of a large dataset so I'm going to remove those
ad = ad.dropna()


In [169]:
###Now I have finished cleaning the Data and I have 10 columns with 70k accidends of information that I'm interested in 

print('Sum of Missing Values in Each Column:  ', ad.isna().sum())
print('Shape of the dataset:  ', ad.shape)

Sum of Missing Values in Each Column:   Accident.Number           0
Aircraft.damage           0
Aircraft.Category         0
Number.of.Engines         0
Engine.Type               0
Total.Fatal.Injuries      0
Total.Serious.Injuries    0
Total.Minor.Injuries      0
Total.Uninjured           0
dtype: int64
Shape of the dataset:   (22253, 9)


In [173]:
""" First, I'm charged with determining which aircraft are the lowest risk for the company. 
Therefore, I grouped the data by aircraft category to see which of the aircraft types has 
the lowest risk"""

###TODO: Calculate the percentage of passengers who aren't injured by the accident to claim which type of aircraft is the safest

ad.groupby(['Aircraft.Category']).count()


,Accident.Number,Aircraft.damage,Number.of.Engines,Engine.Type,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured
Aircraft.Category,,,,,,,,
Airplane,19371,19371,19371,19371,19371,19371,19371,19371
Balloon,32,32,32,32,32,32,32,32
Blimp,4,4,4,4,4,4,4,4
Glider,115,115,115,115,115,115,115,115
Gyrocraft,25,25,25,25,25,25,25,25
Helicopter,2494,2494,2494,2494,2494,2494,2494,2494
Powered Parachute,75,75,75,75,75,75,75,75
Powered-Lift,2,2,2,2,2,2,2,2
Rocket,1,1,1,1,1,1,1,1


In [174]:
ad = ad[ad['Aircraft.Category'] == 'Airplane']

In [177]:
ad.groupby(['Number.of.Engines', 'Engine.Type']).sum()

Total.Fatal.Injuries  \
Number.of.Engines Engine.Type                             
0.0               Reciprocating                     0.0   
                  Unknown                           1.0   
1.0               Electric                          0.0   
                  Reciprocating                  4053.0   
                  Turbo Fan                         8.0   
                  Turbo Jet                        15.0   
                  Turbo Prop                      335.0   
                  Turbo Shaft                       0.0   
                  UNK                               0.0   
                  Unknown                           6.0   
2.0               Geared Turbofan                   0.0   
                  Reciprocating                  1054.0   
                  Turbo Fan                       579.0   
                  Turbo Jet                        39.0   
                  Turbo Prop                      486.0   
                  Turbo Shaft                       3.0   
                  Unknown                          10.0   
3.0               Turbo Fan                         4.0   
                  Turbo Jet                         0.0   
4.0               Electric                          0.0   
                  Reciprocating                    12.0   
                  Turbo Fan                       231.0   
                  Turbo Jet                        18.0   
                  Turbo Prop                        0.0   
8.0               Electric                          0.0   

                                   Total.Serious.Injuries  \
Number.of.Engines Engine.Type                               
0.0               Reciprocating                       0.0   
                  Unknown                             0.0   
1.0               Electric                            0.0   
                  Reciprocating                    3371.0   
                  Turbo Fan                           0.0   
                  Turbo Jet                           8.0   
                  Turbo Prop                        158.0   
                  Turbo Shaft                         0.0   
                  UNK                                 0.0   
                  Unknown                             2.0   
2.0               Geared Turbofan                     0.0   
                  Reciprocating                     326.0   
                  Turbo Fan                         536.0   
                  Turbo Jet                          31.0   
                  Turbo Prop                        101.0   
                  Turbo Shaft                         0.0   
                  Unknown                             4.0   
3.0               Turbo Fan                          24.0   
                  Turbo Jet                           0.0   
4.0               Electric                            0.0   
                  Reciprocating                       2.0   
                  Turbo Fan                          54.0   
                  Turbo Jet                           0.0   
                  Turbo Prop                          0.0   
8.0               Electric                            0.0   

                                   Total.Minor.Injuries  Total.Uninjured  
Number.of.Engines Engine.Type                                             
0.0               Reciprocating                     0.0              1.0  
                  Unknown                           0.0              3.0  
1.0               Electric                          0.0              2.0  
                  Reciprocating                  2801.0          17952.0  
                  Turbo Fan                         2.0              8.0  
                  Turbo Jet                         3.0             18.0  
                  Turbo Prop                      132.0            898.0  
                  Turbo Shaft                       2.0              8.0  
                  UNK                      

In [49]:
ad.shape

(88958, 29)

Event.Id                     69
Investigation.Type            0
Accident.Number              69
Event.Date                   69
Location                    121
Country                     295
Airport.Code              38709
Airport.Name              36168
Injury.Severity            1069
Aircraft.damage            3263
Aircraft.Category         56671
Registration.Number        1386
Make                        132
Model                       161
Amateur.Built               171
Number.of.Engines          6153
Engine.Type                7146
FAR.Description           56935
Schedule                  76376
Purpose.of.flight          6261
Air.carrier               72310
Total.Fatal.Injuries      11470
Total.Serious.Injuries    12579
Total.Minor.Injuries      12002
Total.Uninjured            5981
Weather.Condition          4561
Broad.phase.of.flight     27234
Report.Status              6450
Publication.Date          15299
dtype: int64

In [23]:
ad[ad.duplicated(keep=False)].sort_values(by='Investigation.Type')

,Event.Id,Investigation.Type,Accident.Number,Event.Date,Location,Country,Latitude,Longitude,Airport.Code,Airport.Name,...,Purpose.of.flight,Air.carrier,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,Weather.Condition,Broad.phase.of.flight,Report.Status,Publication.Date
85351,NaN,01-06-2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
86122,NaN,01-06-2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
86140,NaN,01-06-2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
86119,NaN,01-06-2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
86154,NaN,01-06-2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86349,NaN,31-08-2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
85193,NaN,31-08-2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
86717,NaN,31-08-2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
86561,NaN,31-08-2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Data Preparation

In [7]:
ad.isna().sum() #checked for missing data

Event.Id                   1459
Investigation.Type            0
Accident.Number            1459
Event.Date                 1459
Location                   1511
Country                    1685
Latitude                  55966
Longitude                 55975
Airport.Code              40099
Airport.Name              37558
Injury.Severity            2459
Aircraft.damage            4653
Aircraft.Category         58061
Registration.Number        2776
Make                       1522
Model                      1551
Amateur.Built              1561
Number.of.Engines          7543
Engine.Type                8536
FAR.Description           58325
Schedule                  77766
Purpose.of.flight          7651
Air.carrier               73700
Total.Fatal.Injuries      12860
Total.Serious.Injuries    13969
Total.Minor.Injuries      13392
Total.Uninjured            7371
Weather.Condition          5951
Broad.phase.of.flight     28624
Report.Status              7840
Publication.Date          16689
dtype: i

In [17]:
ad['Make'].value_counts()

Cessna                           22227
Piper                            12029
CESSNA                            4922
Beech                             4330
PIPER                             2841
                                 ...  
William F. Lair                      1
AIRGYRO                              1
Quad City Ultralight                 1
QUARTZ MOUNTAIN AEROSPACE INC        1
Harold Vander                        1
Name: Make, Length: 8237, dtype: int64

In [18]:
ad['Model'].value_counts()


152                     2367
172                     1756
172N                    1164
PA-28-140                932
150                      829
                        ... 
KCA                        1
HOFFMAN BMW AIRTRIKE       1
P-2                        1
300A                       1
MIRAGE F1 B                1
Name: Model, Length: 12318, dtype: int64

Exploratory Data Analysis

Conclusions

Limitations

Recommendations

Next Steps